In [123]:
# Forecasting Snow Depth with Prophet


## Abstract
[Prophet](https://facebook.github.io/prophet/) is a great tool for forecasting time series data based on an additive model where non-linear trends are fitted with yearly, weekly, and daily seasonality, plus holiday effects. Considering climate change, I was wondering if this tool can be used to predict snow depth evolution in the French Alps and particularly in the French ski resorts. Obviously, this is not scientifically significant since weather prediction is a sophisticated task based on the Lorenz equations that govern the weather system but it is clearly fun to play with these data.

## Data Analysis
In France, snow report data of 135 ski resorts are available freely from [Meteo France](https://donneespubliques.meteofrance.fr/?fond=produit&id_produit=94&id_rubrique=32). These data are available as csv files and give several observations (wind direction and strength, air temperature, humidity, snow depths and various parameters for characterizing snow accumulated on the ground) from the different snow sensors every 12 hours. The following map constructed with [Folium](https://python-visualization.github.io/folium) exhibits the locations of the different available snow sensors. Among them, famous ski resorts such as Val d'Isère or l'Alpe d'Huez can be retrieved.

In [124]:
import folium

m = folium.Map(width='100%', height='100%', location=[48.856578, 2.351828], zoom_start=6, tiles='Stamen Terrain',
               attr='Snow Sensors')
folium.GeoJson('../data/postesNivo.json', name='geojson').add_to(m)
m


Meteo France provides a file per month since 2010 with measurements coming from each of these snow sensors. As an example, let's load the file containing observations for january 2018 in a dataframe and prints its content.

In [125]:
import pandas

measurements_201801 = pandas.read_csv('../data/nivo.201801.csv', delimiter=';', parse_dates=['date'])
measurements_201801.head()

,numer_sta,date,haut_sta,dd,ff,t,td,u,ww,w1,...,ht_neige_alti,neige_fraiche,teneur_eau,grain_predom,grain_nombre,grain_diametr,homogeneite,m_vol_neige,Unnamed: 48,Unnamed: 49
0,7590,2018-01-01 07:10:00,1270.0,0,0.000000,274.250000,273.540000,95,0,7,...,mq,mq,mq,mq,mq,mq,mq,mq,NaN,NaN
1,7818,2018-01-01 07:10:00,1250.0,360,2.000000,271.150000,268.160000,80,0,7,...,mq,mq,mq,mq,mq,mq,mq,mq,NaN,NaN
2,7856,2018-01-01 06:30:00,1620.0,0,0.000000,270.350000,269.660000,95,70,0,...,3.100000,0.200000,mq,1,9,mq,0,80.000000,NaN,NaN
3,7888,2018-01-01 06:15:00,1800.0,0,0.000000,269.350000,267.950000,90,0,7,...,mq,mq,mq,1,2,mq,0,100.000000,NaN,NaN
4,7895,2018-01-01 06:40:00,1970.0,0,0.000000,268.150000,267.330000,94,70,7,...,mq,mq,mq,1,1,mq,mq,mq,NaN,NaN


As previously emphasized, several measures are given every 12 hours such as :
* haut_sta : the altitude of the observation ;
* t : the temperature (in Kelvin) ;
* neige_fraiche : the quantity of new snow (in meters) since the last observations ;
* ht_neige : the quantity of snow (in meters).

The latitude, longitude, altitude, name of each snow sensors are also provided in a csv file and can thus be used to obtain the location of the measurements. Let's load this file in a dataframe and prints its content.

In [126]:
stations = pandas.read_csv('../data/postesNivo.csv', delimiter=',')
stations.head()

,Latitude,Longitude,ID,Altitude,Nom
0,46.766000,6.359167,7392.0,1036,METABIEF_STATION
1,46.020833,6.970833,7393.0,2196,LE TOUR BALME
2,46.341167,6.708167,7454.0,1535,Bernex
3,45.247667,6.732667,7456.0,2166,Aussois
4,46.315333,6.673333,7457.0,790,VACHERESSE AUXI


We propose to aggregate all measurements in a dataframe and to join them with the list of snow sensors in order to put all the observations of all stations in a single dataframe.

In [127]:
import glob

path = '../data/'
nivo_files = [f for f in glob.glob(path + 'nivo*.csv')]
li = [pandas.read_csv(filename, delimiter=';', parse_dates=['date']) for filename in nivo_files]
finalMeasurements = pandas.concat(li, ignore_index=True)

If one wants to list all observations for [Tignes](https://www.tignes.net/) ski resort, pattern matching on the ski resort name is enough.

In [128]:
measurements = finalMeasurements.loc[finalMeasurements.numer_sta == int(stations.loc[stations.Nom == 'Tignes']['ID'])]
measurements

,numer_sta,date,haut_sta,dd,ff,t,td,u,ww,w1,...,ht_neige_alti,neige_fraiche,teneur_eau,grain_predom,grain_nombre,grain_diametr,homogeneite,m_vol_neige,Unnamed: 48,Unnamed: 49
21,7904,2010-12-01 07:00:00,2080.0,360,2.000000,267.650000,265.980000,88,70,7,...,mq,mq,mq,2,2,mq,mq,mq,NaN,NaN
41,7904,2010-12-01 12:00:00,2080.0,0,0.000000,268.550000,265.460000,79,70,7,...,mq,mq,mq,1,2,mq,mq,mq,NaN,NaN
64,7904,2010-12-02 07:00:00,2080.0,0,0.000000,257.150000,249.970000,54,0,7,...,mq,mq,mq,1,1,mq,0,50.000000,NaN,NaN
85,7904,2010-12-02 12:00:00,2080.0,0,0.000000,263.350000,243.440000,18,0,0,...,mq,mq,mq,1,2,mq,mq,mq,NaN,NaN
116,7904,2010-12-03 07:00:00,2080.0,0,0.000000,257.650000,251.640000,60,0,0,...,mq,mq,mq,2,2,mq,mq,mq,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233116,7904,2021-12-28 06:00:00,2080.0,0,0.000000,271.050000,270.780000,98,70,7,...,mq,mq,mq,1,1,mq,0,70.000000,NaN,NaN
233259,7904,2021-12-29 05:30:00,2080.0,90,2.000000,271.550000,271.410000,99,75,7,...,mq,mq,mq,1,1,mq,mq,mq,NaN,NaN
233319,7904,2021-12-29 12:30:00,2080.0,0,0.000000,273.250000,273.110000,99,69,7,...,mq,mq,mq,1,1,mq,mq,mq,NaN,NaN
233399,7904,2021-12-30 09:30:00,2080.0,0,0.000000,271.950000,271.670000,98,0,6,...,mq,mq,mq,6,2,mq,mq,mq,NaN,NaN


## Forecasting Snow Depth
The dataframe is used to train the prophet model and to predict daily snow depth (ht_neige) for the next five years for [Tignes](https://www.tignes.net/) ski resort. A logistic growth trend model with a lower cap of 0, a higher cap of 5 and a yearly seasonality is used. Indeed, we have not observed any patterns at week and daily levels when analysing the data. The results are visualized with [Plotly](https://plotly.com/python/).

In [ ]:
from prophet import Prophet
from plotly import graph_objs as go
from plotly.offline import init_notebook_mode
import plotly.offline as py
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

init_notebook_mode(connected=True)

TARGET_KEY = 'ht_neige'
SKI_RESORT = 'Tignes'
CAP = 5
FLOOR = 0

# Data preparation
dailyDataframe = finalMeasurements.loc[
    finalMeasurements.numer_sta == int(stations.loc[stations.Nom == SKI_RESORT]['ID'])]
dailyDataframe = dailyDataframe[dailyDataframe[TARGET_KEY] != 'mq']  # Remove missing values

dailyDataframe['date'] = pandas.to_datetime(dailyDataframe['date'])
dailyDataframe[TARGET_KEY] = pandas.to_numeric(dailyDataframe[TARGET_KEY], errors='ignore')

# Detect start and end date of measurements every ski seasons
years = dailyDataframe['date'].dt.year.unique()
minMaxMeasurementDate = []
for year in sorted(years[:-1]):
    minMaxMeasurementDate.append((dailyDataframe[dailyDataframe['date'] > f'{year}-09-01']['date'].min(),
                                  dailyDataframe[dailyDataframe['date'] < f'{year + 1}-06-01']['date'].max()))

# Interpolate missing values within the season
dailyDataframe = dailyDataframe.set_index('date').resample('D').mean().interpolate()

# Fill with 0 when outside measurement range
dailyDataframe.reset_index(drop=False, inplace=True)
for index in range(1, len(minMaxMeasurementDate)):
    dailyDataframe.loc[
        ((minMaxMeasurementDate[index][0] > dailyDataframe.date) & (
                    dailyDataframe.date > minMaxMeasurementDate[index - 1][1])), TARGET_KEY] = 0

dailyDataframe = dailyDataframe[['date', TARGET_KEY]]
dailyDataframe = dailyDataframe.rename(columns={'date': 'ds', TARGET_KEY: 'y'})

# Model fitting and forecasting
df0 = dailyDataframe.copy()
dailyDataframe = dailyDataframe[:-365]
prophetModelDataframe = Prophet(growth='logistic', daily_seasonality=False, weekly_seasonality=False)
dailyDataframe['cap'] = CAP
dailyDataframe['floor'] = FLOOR
prophetModelDataframe.fit(dailyDataframe)
forecastedDataframe = prophetModelDataframe.make_future_dataframe(periods=1825)
forecastedDataframe['cap'] = CAP
forecastedDataframe['floor'] = FLOOR
forecastedDataframe = prophetModelDataframe.predict(forecastedDataframe)

# Results visualisation
trace = go.Scatter(
    name='Observed Snow Depth (used for training the model)',
    mode='markers',
    x=list(forecastedDataframe['ds']),
    y=list(dailyDataframe['y']),
    marker=dict(
        color='#FFBAD2',
        line=dict(width=1)
    )
)

trace1 = go.Scatter(
    name='trend',
    mode='lines',
    x=list(forecastedDataframe['ds']),
    y=list(forecastedDataframe['yhat']),
    marker=dict(
        color='red',
        line=dict(width=3)
    )
)

upper_band = go.Scatter(
    name='upper band',
    mode='lines',
    x=list(forecastedDataframe['ds']),
    y=list(forecastedDataframe['yhat_upper']),
    line=dict(color='#57b88f'),
    fill='tonexty'
)

lower_band = go.Scatter(
    name='lower band',
    mode='lines',
    x=list(forecastedDataframe['ds']),
    y=list(forecastedDataframe['yhat_lower']),
    line=dict(color='#1705ff')
)

tracex = go.Scatter(
    name='Observed Snow Depth (not used for training the model)',
    mode='markers',
    x=list(df0['ds']),
    y=list(df0['y']),
    marker=dict(
        color='black',
        line=dict(width=2)
    )
)

data = [tracex, trace1, lower_band, upper_band, trace]

layout = dict(title=f'Snow Depth Forecasting Using Prophet for {SKI_RESORT}',
              xaxis=dict(title='Dates', ticklen=2, zeroline=True))

figure = dict(data=data, layout=layout)
py.offline.iplot(figure)

Initial log joint probability = -131.234
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       6701.96    0.00453921        113.85           1           1      117   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     146        6705.6    0.00125446       92.4445   8.416e-06       0.001      217  LS failed, Hessian reset 
     163        6706.1   0.000116468       47.6713    3.07e-06       0.001      274  LS failed, Hessian reset 
     197       6708.09    0.00656109       187.271   5.833e-05       0.001      363  LS failed, Hessian reset 
     199       6708.52    0.00205029       106.854      0.9029      0.9029      365   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     230       6709.25   0.000848532       76.8229   1.148e-05       0.001      447  LS failed, Hessian reset 
     294       6710.72    0.00464845       193.675   5.634e-05   

From the above plot of forecast, we can see that the forecast follows the local trend of actual values. Moreover, it can be observed that the trend remains constant from 2011 to 2018 and then decreases till 2018. Finally, it is worth noting that the lower band should be capped at 0 since it is not possible to have a negative snow depth.

## Using the Forecast to Predict Ski Season Length
For a ski resort, it could be interesting to know the length of the ski season (the number of day it will be possible to ski thanks to snow depth). if we suppose than it needs more than 0.3 meter of snow to ski then it is easy to compute start date, end date and length of the yearly ski seasons.

In [ ]:
SNOW_DEPTH_THRESHOLD = 0.3
skiDayDataframe = forecastedDataframe[['ds', 'yhat']]
skiDayDataframe = skiDayDataframe[skiDayDataframe['yhat'] > SNOW_DEPTH_THRESHOLD]
years = skiDayDataframe['ds'].dt.year.unique()
for year in years[:-1]:
    startDate = skiDayDataframe[skiDayDataframe['ds'] > f'{year}-09-01']['ds'].min()
    endDate = skiDayDataframe[skiDayDataframe['ds'] < f'{year + 1}-06-01']['ds'].max()
    print(
        f'Season {year}/{year + 1}: Start date: {startDate.year}-{startDate.month}-{startDate.day}, End date: {endDate.year}-{endDate.month}-{endDate.day}, Season length: {(endDate - startDate).days} days')